In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
champ_data = pd.read_excel("LeagueChampsData.xlsx")

In [3]:
SUPPORT_ITEM_LIST = ["Bloodsong","Celestial Opposition","Dream Maker","Solstice Sleigh","Zaz'Zak's Realmspike"]
LEGENDARY_ITEM_LIST = ["Abyssal Mask","Archangel's Staff","Ardent Censer","Axiom Arc","Banshee's Veil","Black Cleaver",
                      "Blackfire Torch","Blade of the Ruined King","Bloodletter's Curse","Bloodthirster","Chempunk Chainsword",
                      "Cosmic Drive","Cryptbloom","Dawncore","Dead Man's Plate","Death's Dance","Echoes of Helia","Eclipse","Edge of Night",
                      "Essence Reaver","Experimental Hexplate","Fimbulwinter","Force of Nature","Frozen Heart","Guardian Angel","Guinsoo's Rageblade",
                      "Heartsteel","Hextech Rocketbelt","Hollow Radiance","Horizon Focus","Hubris","Hullbreaker","Iceborn Gauntlet",
                       "Immortal Shieldbow","Imperial Mandate","Infinity Edge","Jak'Sho, The Protean","Kaenic Rookern","Knight's Vow",
                      "Kraken Slayer","Liandry's Torment","Lich Bane","Locket of the Iron Solari","Lord Dominik's Regards","Luden's Companion",
                      "Malignance","Manamune","Maw of Malmortius","Mejai's Soulstealer","Mercurial Scimitar","Mikael's Blessing",
                      "Moonstone Renewer","Morellonomicon","Mortal Reminder","Muramana","Nashor's Tooth","Navori Flickerblade",
                      "Opportunity","Overlord's Bloodmail","Phantom Dancer","Profane Hydra","Rabadon's Deathcap","Randiun's Omen",
                      "Rapid Firecannon","Ravenous Hydra","Redemption","Riftmaker","Rod of Ages","Runaan's Hurricane","Rylai's Crystal Scepter",
                      "Seraph's Embrace","Serpent's Fang","Serylda's Grudge","Shadowflame","Shurelya's Battlesong","Spear of Shojin",
                      "Spirit's Visage","Staff of Flowing Water","Statikk Shiv","Sterak's Gage","Stormsurge","Stridebreaker",
                      "Sundered Sky","Sunfire Aegis","Terminus","The Collector","Thornmail","Titanic Hydra","Trailblazer","Trinity Force",
                      "Umbral Glaive","Unending Despair","Vigilant Wardstone","Void Staff","Voltaic Cyclosword","Warmog's Armor",
                      "Winter's Approach","Wit's End","Youmuu's Ghostblade","Yun Tal Wildarrows","Zeke's Convergence","Zhonya's Hourglass"]

In [4]:
SUPPORT_ITEM_LIST = [item.lower() for item in SUPPORT_ITEM_LIST]
LEGENDARY_ITEM_LIST = [item.lower() for item in LEGENDARY_ITEM_LIST]

In [5]:
all_champs = champ_data["Champion"].tolist()

In [6]:
num_champs = len(all_champs)
num_items = len(LEGENDARY_ITEM_LIST)

In [7]:
champ_dict = {champ.lower():i for i,champ in enumerate(all_champs)}
item_dict = {item:i for i, item in enumerate(LEGENDARY_ITEM_LIST)}

In [8]:
match_data = pd.read_csv("league_matches.csv")

In [9]:
item_list = match_data["items"]

In [10]:
champs_list = match_data["enemy champs"].tolist()

### Convert items and champions from string to list

In [11]:
legendary_items = []
for match in item_list:
    match_items = []
    match = list(eval(match))
    for item in match:
        if item.lower() in LEGENDARY_ITEM_LIST:
            match_items.append(item.lower())
    legendary_items.append(match_items)

In [12]:
enemy_champs = []
for champ in champs_list:
    champions = list(eval(champ))
    enemy_champs.append(champions)

In [13]:
match_data['complete items'] = legendary_items

In [14]:
match_data['enemy champs'] = enemy_champs

### Make sure champs are Bot-Lane Champs (until all champs are added)

In [15]:
new_df = match_data[match_data["enemy champs"].apply(lambda lst: all(c in all_champs for c in lst))]

### Convert champs and items to IDs

In [16]:
final_df = pd.DataFrame()
final_df['complete items'] = new_df['complete items']
final_df['enemy champs'] = new_df['enemy champs']
final_df['enemy champ ids'] = final_df["enemy champs"].apply(lambda lst: [champ_dict[lst[0].lower()],champ_dict[lst[1].lower()]])
final_df['item ids'] = final_df["complete items"].apply(lambda lst: [1 if item in lst else 0 for item in LEGENDARY_ITEM_LIST])

In [17]:
id_df = final_df[['enemy champ ids','item ids']]

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

In [19]:
class BuildReccomender(nn.Module):
    def __init__(self, num_champs, embed_dim, hidden_dim, output):
        super(BuildReccomender, self).__init__()
        self.embed = nn.Embedding(num_champs, embed_dim)
        
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output)

    def forward(self, enemy_champs):
        champion_embeddings = self.embed(enemy_champs)
        x = champion_embeddings.mean(dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [21]:
X = torch.tensor(id_df['enemy champ ids'].tolist(), dtype=torch.long).to(device)
y = torch.tensor(id_df['item ids'].tolist()).to(device).float()

In [22]:
embed_dim = 64
hidden_dim = 64

In [23]:
model = BuildReccomender(num_champs, embed_dim, hidden_dim, num_items)
model.to(device)

BuildReccomender(
  (embed): Embedding(55, 64)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=102, bias=True)
)

In [24]:
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [25]:
epochs = 500
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 388.66it/s]


### Test model

In [26]:
enemy_adc = input("Enemy ADC: ").lower()
enemy_sup = input("Enemy Support: ").lower()
enemy_adc = champ_dict[enemy_adc]
enemy_sup = champ_dict[enemy_sup]

Enemy ADC:  samira
Enemy Support:  lux


In [27]:
model.eval()
with torch.no_grad():
    enemy_input = torch.tensor([[enemy_adc, enemy_sup]], dtype=torch.long).to(device)
    
    preds = model(enemy_input)

In [28]:
probs = preds.tolist()[0]
item_prob = {}
for i in range(num_items):
    item_prob[LEGENDARY_ITEM_LIST[i]] = probs[i]

In [29]:
sorted_item_probs = sorted(item_prob.items(), key = lambda item: item[1], reverse = True)

In [30]:
sorted_item_probs

[('heartsteel', 0.5335891246795654),
 ('iceborn gauntlet', 0.23879648745059967),
 ('thornmail', 0.12378160655498505),
 ('force of nature', 0.0793798416852951),
 ('rapid firecannon', 0.07822369784116745),
 ("dead man's plate", 0.07355822622776031),
 ("warmog's armor", 0.0656617283821106),
 ('kraken slayer', 0.058671269565820694),
 ("wit's end", 0.0578019842505455),
 ('staff of flowing water', 0.054629210382699966),
 ('locket of the iron solari', 0.05447912588715553),
 ('kaenic rookern', 0.04901741445064545),
 ('infinity edge', 0.044135842472314835),
 ('redemption', 0.04390406608581543),
 ('umbral glaive', 0.04271405562758446),
 ('moonstone renewer', 0.042525921016931534),
 ('sunfire aegis', 0.04109103977680206),
 ('bloodthirster', 0.03792892023921013),
 ('frozen heart', 0.035855088382959366),
 ('lich bane', 0.03467665985226631),
 ('ravenous hydra', 0.034626226872205734),
 ('unending despair', 0.03455892950296402),
 ('riftmaker', 0.0341116264462471),
 ("zeke's convergence", 0.03410771489

### Save model

In [32]:
torch.save(model.state_dict(),"model.pth")